In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [99]:
train_df = pd.read_csv('./fe_v2_train_076.csv')
test_df = pd.read_csv('./fe_v2_test_076.csv')

In [110]:
exclude_cols = ['index', 'startdate']
embedding_attrs = ['climateregions__climateregion']
target=["contest-tmp2m-14d__tmp2m"]

In [111]:
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 129


In [101]:
group_cols = ['loc_group', 'climate_type', 'year', 'season', 'month']
season_group_cols = ['loc_group', 'climate_type', 'year', 'season']

In [106]:
best_cols = ['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'nmme-prate-34w__cancm4',
 'sst-2010-4',
 'nmme0-prate-56w__ccsm30',
 'wind-uwnd-250-2010-16',
 'nmme0-prate-34w__gfdl0',
 'nmme0-prate-56w__cancm40',
 'sst-2010-1',
 'sst-2010-3',
 'wind-uwnd-250-2010-14',
 'nmme0-prate-34w__gfdlflora0',
 'nmme-prate-34w__gfdl',
 'wind-hgt-850-2010-9',
 'wind-vwnd-250-2010-1',
 'sst-2010-5',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__nasa0',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__cancm30',
 'wind-vwnd-250-2010-13',
 'wind_diff',
 'wind_diff_min',
 'wind_diff_min_month',
 'wind_diff_month',
 'diff_slp_first',
 'diff_wind_first',
 'diff_precip_first',
 'diff_sst_10_first',
 'diff_sst_10_min',
 'diff_sst_10_max',
 'range_sst_10',
 'scale_sst_10',
 'diff_sst_9_first',
 'diff_sst_9_min',
 'diff_sst_9_max',
 'range_sst_9',
 'scale_sst_9',
 'diff_sst_8_first',
 'diff_sst_8_min',
 'diff_sst_8_max',
 'range_sst_8',
 'scale_sst_8',
 'diff_sst_7_first',
 'diff_sst_7_min',
 'diff_sst_7_max',
 'range_sst_7',
 'scale_sst_7',
 'diff_sst_6_first',
 'diff_sst_6_min',
 'diff_sst_6_max',
 'range_sst_6',
 'scale_sst_6',
 'diff_sst_1_first',
 'diff_sst_1_min',
 'diff_sst_1_max',
 'range_sst_1',
 'scale_sst_1',
 'diff_sst_2_first',
 'diff_sst_2_min',
 'diff_sst_2_max',
 'range_sst_2',
 'scale_sst_2',
 'diff_sst_3_first',
 'diff_sst_3_min',
 'diff_sst_3_max',
 'range_sst_3',
 'scale_sst_3',
 'diff_sst_4_first',
 'diff_sst_4_min',
 'diff_sst_4_max',
 'range_sst_4',
 'scale_sst_4',
 'diff_sst_5_first',
 'diff_sst_5_min',
 'diff_sst_5_max',
 'range_sst_5',
 'scale_sst_5']

In [107]:
print("Len:", len(best_cols))

Len: 116


In [108]:
def assign_train_test_df(df, my_col):
    train_df[my_col] = df[:len(train_df)][my_col].tolist()
    test_df[my_col] = df[len(train_df):][my_col].tolist()
    return df

def create_df_by_group_and_agg(dataframe, grouped_cols, action, my_col, return_name):
    if action == "first":
        ans_df = dataframe.groupby(grouped_cols)[my_col].first().reset_index()
    elif action == "min":
        ans_df = dataframe.groupby(grouped_cols)[my_col].min().reset_index()
    elif action == "max":
        ans_df = dataframe.groupby(grouped_cols)[my_col].max().reset_index()
    elif action == "mean":
        ans_df = dataframe.groupby(grouped_cols)[my_col].mean().reset_index()
    print("Nan")
    ans_df = ans_df.rename(columns={my_col: return_name})
    for c in ans_df.columns:
        if ans_df[c].isnull().sum() > 0:
            print(c, ans_df[c].isnull().sum())
    return ans_df

def create_feature(df, alias_name, col_name, suffix_num, season_group_cols):
    first_df = create_df_by_group_and_agg(df, season_group_cols, "first", col_name, f"first_{alias_name}_{suffix_num}")
    min_df = create_df_by_group_and_agg(df, season_group_cols, "min", col_name, f"min_{alias_name}_{suffix_num}")
    max_df = create_df_by_group_and_agg(df, season_group_cols, "max", col_name, f"max_{alias_name}_{suffix_num}")
    
    df = df.merge(first_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_first'] = df[col_name] - df[f"first_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_first')
    
    df = df.merge(min_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_min'] = df[col_name] - df[f"min_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_min')
    
    df = df.merge(max_df, how='left', on=season_group_cols)
    df[f'diff_{alias_name}_{suffix_num}_max'] = df[col_name] - df[f"max_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'diff_{alias_name}_{suffix_num}_max')
    
    df[f'range_{alias_name}_{suffix_num}'] = df[f"max_{alias_name}_{suffix_num}"] - df[f"min_{alias_name}_{suffix_num}"]
    df = assign_train_test_df(df, f'range_{alias_name}_{suffix_num}')
    
    df[f'scale_{alias_name}_{suffix_num}'] = (df[col_name] - df[f"min_{alias_name}_{suffix_num}"]) / df[f'range_{alias_name}_{suffix_num}']
    df = assign_train_test_df(df, f'scale_{alias_name}_{suffix_num}')
    
    return [df, f'diff_{alias_name}_{suffix_num}_first',
   f'diff_{alias_name}_{suffix_num}_min',
    f'diff_{alias_name}_{suffix_num}_max',
    f'range_{alias_name}_{suffix_num}', f'scale_{alias_name}_{suffix_num}']

In [14]:
slp_col = 'contest-slp-14d__slp'
ccsm3_col = 'nmme-tmp2m-34w__ccsm3'
pres_col = 'contest-pres-sfc-gauss-14d__pres'
precip_col = 'contest-precip-14d__precip'
poten_col = 'contest-pevpr-sfc-gauss-14d__pevpr'
rhum_col = 'contest-rhum-sig995-14d__rhum'
elevation_col = 'elevation__elevation'
sst_1_col = ''

In [112]:
df = pd.concat([train_df[main_attrs], test_df[main_attrs]], axis=0)
df.shape

(407088, 129)

In [113]:
pres_res = create_feature(df, 'pres', pres_col, 1, season_group_cols)
df = pres_res[0]
pres_feature = pres_res[1:]

Nan
Nan
Nan


In [116]:
features = best_cols + pres_feature

In [117]:
X = train_df[features].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=200, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[features].values))
s = 0
for pred in preds:
    s += pred
s = s/5
res_df = pd.read_csv('./sample_solution.csv')
res_df['contest-tmp2m-14d__tmp2m'] = s
res_df.to_csv('submission_fe_v5.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

5it [00:01,  4.48it/s]


(375734, 121) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.072699
0:	learn: 9.2611105	test: 9.2407165	best: 9.2407165 (0)	total: 6.15ms	remaining: 30.7s
200:	learn: 1.1730213	test: 1.1790287	best: 1.1790287 (200)	total: 992ms	remaining: 23.7s
400:	learn: 0.9201555	test: 0.9288454	best: 0.9288454 (400)	total: 1.95s	remaining: 22.4s
600:	learn: 0.7895557	test: 0.8004377	best: 0.8004377 (600)	total: 2.94s	remaining: 21.5s
800:	learn: 0.7014437	test: 0.7139431	best: 0.7139431 (800)	total: 3.93s	remaining: 20.6s
1000:	learn: 0.6436207	test: 0.6572365	best: 0.6572365 (1000)	total: 4.96s	remaining: 19.8s
1200:	learn: 0.5989952	test: 0.6139161	best: 0.6139161 (1200)	total: 5.95s	remaining: 18.8s
1400:	learn: 0.5641052	test: 0.5803106	best: 0.5803106 (1400)	total: 6.99s	remaining: 18s
1600:	learn: 0.5338744	test: 0.5510697	best: 0.5510697 (1600)	total: 8.03s	remaining: 17s
1800:	learn: 0.5083123	test: 0.5264594	best: 0.5264594 (1800)	total: 9.07s	remaining: 16.1s
2000:	learn: 0.4870850	test: 0.5061276	best: 0.5061276 (2000)	total

 20%|█████████████████▌                                                                      | 1/5 [00:36<02:27, 36.98s/it]

...

Learning rate set to 0.072699
0:	learn: 9.2604847	test: 9.2566518	best: 9.2566518 (0)	total: 7.15ms	remaining: 35.8s
200:	learn: 1.1668541	test: 1.1809546	best: 1.1809546 (200)	total: 1.02s	remaining: 24.4s
400:	learn: 0.9057562	test: 0.9202933	best: 0.9202933 (400)	total: 2s	remaining: 23s
600:	learn: 0.7795407	test: 0.7942298	best: 0.7942298 (600)	total: 3.02s	remaining: 22.1s
800:	learn: 0.6985402	test: 0.7139726	best: 0.7139726 (800)	total: 4.05s	remaining: 21.2s
1000:	learn: 0.6386407	test: 0.6550158	best: 0.6550158 (1000)	total: 4.98s	remaining: 19.9s
1200:	learn: 0.5937537	test: 0.6109614	best: 0.6109614 (1200)	total: 5.99s	remaining: 18.9s
1400:	learn: 0.5592268	test: 0.5776496	best: 0.5776496 (1400)	total: 6.97s	remaining: 17.9s
1600:	learn: 0.5297275	test: 0.5492935	best: 0.5492935 (1600)	total: 8s	remaining: 17s
1800:	learn: 0.5052976	test: 0.5253812	best: 0.5253812 (1800)	total: 8.96s	remaining: 15.9s
2000:	learn: 0.4836605	test: 0.5045818	best: 0.5045818 (2000)	total: 9.9s

 80%|██████████████████████████████████████████████████████████████████████▍                 | 4/5 [02:30<00:37, 37.65s/it]

Learning rate set to 0.072699
0:	learn: 9.2625918	test: 9.2370575	best: 9.2370575 (0)	total: 6.32ms	remaining: 31.6s
200:	learn: 1.1674422	test: 1.1729522	best: 1.1729522 (200)	total: 973ms	remaining: 23.2s
400:	learn: 0.9112754	test: 0.9196302	best: 0.9196302 (400)	total: 1.95s	remaining: 22.4s
600:	learn: 0.7841273	test: 0.7941229	best: 0.7941229 (600)	total: 2.9s	remaining: 21.3s
800:	learn: 0.7044739	test: 0.7158356	best: 0.7158356 (800)	total: 3.81s	remaining: 20s
1000:	learn: 0.6445724	test: 0.6574419	best: 0.6574419 (1000)	total: 4.84s	remaining: 19.3s
1200:	learn: 0.6004392	test: 0.6151505	best: 0.6151505 (1200)	total: 5.82s	remaining: 18.4s
1400:	learn: 0.5621739	test: 0.5781922	best: 0.5781922 (1400)	total: 6.85s	remaining: 17.6s
1600:	learn: 0.5321143	test: 0.5494628	best: 0.5494628 (1600)	total: 7.88s	remaining: 16.7s
1800:	learn: 0.5083344	test: 0.5266843	best: 0.5266843 (1800)	total: 8.97s	remaining: 15.9s
2000:	learn: 0.4866637	test: 0.5058550	best: 0.5058550 (2000)	tota

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 20.87it/s]


cos_sim with best submission: 0.9999532551063645
